<a href="https://colab.research.google.com/github/BokyungQQ/Boot_camp_Final_project/blob/main/%EC%9E%AC%EA%B5%AC%EB%A7%A4_%ED%98%84%ED%99%A9_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import hashlib

## 재구매율

In [ ]:
review = pd.read_csv('/content/review.csv')
user = pd.read_csv('/content/user.csv')
review

In [ ]:
review['상품명'].unique()

In [ ]:
review['리뷰작성일시'] = pd.to_datetime(review['리뷰작성일시'])
review['월'] = review['리뷰작성일시'].dt.month
review['월']

0        5
1        5
2        5
3        5
4        5
        ..
2761    11
2762    11
2763    11
2764    11
2765    11
Name: 월, Length: 2766, dtype: int64

비회원 재구매율 확인

In [ ]:
buy = pd.read_csv('/content/order_sales_1206.csv')
buy

In [ ]:
# 비회원
nonmember_count = buy['주문자ID']
nonmember_count = [id for id in nonmember_count if 'nonmember' in id]
nonmember_count = len(nonmember_count)
nonmember_count

4028

In [ ]:
# 비회원 재구매율 확인
( 4028 / 9229 ) * 100

43.64503196445986

In [ ]:
# 주문 내역을 비회원과 회원으로 분리
order_non = buy[buy['주문자ID'].str.contains('nonmember')]
order_join = buy[~buy['주문자ID'].str.contains('nonmember')]

In [ ]:
# 주문자 ID를 생성하는 함수
def generate_customer_id(address):
    hash_object = hashlib.sha256(address.encode())
    return hash_object.hexdigest()[:8]  # 여기서는 앞 8자리만 사용하여 ID 생성
# 주문자 주소를 기반으로 주문자 ID 생성 및 데이터셋에 추가
order_non['비회원 ID'] = order_non['주문자 주소'].apply(generate_customer_id)
# 변경된 데이터셋 확인
order_non = order_non.iloc[:,1:]
order_non

In [ ]:
# 비회원 구매 횟수 확인
non_cnt = order_non.groupby('비회원 ID').size().reset_index(name='cnt')
non_cnt

,비회원 ID,cnt
0,00012312,1
1,000b72ad,1
2,00128686,1
3,0034bf9c,1
4,0037851c,1
...,...,...
3833,ff8c3a0b,1
3834,ffe37b83,1
3835,fff2e405,1
3836,fff427f5,1


In [ ]:
non_cnt.groupby('cnt').size()
# 1회 구매한 비회원이 3666으로 가장 많음

cnt
1    3666
2     155
3      15
4       2
dtype: int64

In [ ]:
# 부트캠프측에서 상세 정보는 삭제하여 집 호수와 같은 세부 정보 없어 같은 아파트면 같은 주소로 작성 ( ex. a 아파트 503호와 a 아파트 203호 모두 a 아파트까지만 작성 )

# '주문자 주소'가 중복되는 경우 중에서 주문자 ID가 다른 경우를 찾기
duplicated_addresses = buy[buy.duplicated('주문자 주소', keep=False)]

# 주문자 ID가 다른 경우만 추출
different_ids = duplicated_addresses[~duplicated_addresses.duplicated('주문자ID')]

# 결과 출력
different_ids

회원가입 후 구매까지 걸린 시간 확인

In [ ]:
# 날짜 형식으로 변환
order_join['주문일시'] = pd.to_datetime(order_join['주문일시'])

# 주문자 ID가 중복된 경우 필터링
duplicated_ids = order_join[order_join.duplicated('주문자ID', keep=False)]

# 주문자 ID가 중복된 경우에만 주문 일시 차이 계산
if not duplicated_ids.empty:
    duplicated_ids['주문 일시 차이'] = duplicated_ids.groupby('주문자ID')['주문일시'].diff()
    average_diff = duplicated_ids.groupby('주문자ID')['주문 일시 차이'].mean()

# 결과 출력
duplicated_ids

In [ ]:
# 주문일시와 주문자 가입일 차이
# differ = order_join[order_join.duplicated('주문자ID', keep='first')]
differ = order_join.sort_values('주문일시').drop_duplicates('주문자ID', keep='first')
differ

In [ ]:
# 주문자 가입일이 비어있는 경우는 5-10월이 아닌 경우이므로 제외
date_nan = differ[~differ['주문자 가입일'].isna()]
# date_nan['주문자ID'].unique().tolist()
date_nan

In [ ]:
# order data  기간 내에 회원 가입한 사람들만 확인
user_register = date_nan[(date_nan['주문자 가입일'] >= '2023-05-01') & (date_nan['주문자 가입일'] < '2023-11-01') ]
user_register

In [ ]:
user_register['주문자 가입일'] = pd.to_datetime(user_register['주문자 가입일'])
user_register['주문일시 - 가입일'] = user_register['주문일시'] - user_register['주문자 가입일']
user_register

In [ ]:
user_register['주문일시 - 가입일'].describe()

count                         4927
mean     1 days 10:35:38.194844733
std      6 days 14:38:37.998981367
min                0 days 00:01:16
25%         0 days 10:49:21.500000
50%                0 days 15:29:57
75%                0 days 21:00:03
max              158 days 10:12:25
Name: 주문일시 - 가입일, dtype: object

## 재구매율 : 구매 패턴

In [ ]:
real_user = pd.read_csv('/content/order_buy_1215.csv')
real_user

In [ ]:
pattern = real_user.groupby(['주문자ID', '주문일시'])['상품명_옵션'].agg(list).reset_index(name='상품명')
pattern['상품명'] = pattern['상품명'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
pattern

In [ ]:
# 주문자 id로 groupby 하고 상품명 분포 확인
pattern = pattern.groupby(['주문자ID', '주문일시'])['상품명'].agg(list).reset_index(name='상품명')
pattern['상품명'] = pattern['상품명'].astype(str)
# pattern['상품명'] = pattern['상품명'].str.replace('[\[\]]', '')
pattern

In [ ]:
pattern = pattern.groupby('주문자ID')['상품명'].agg(list).reset_index(name='상품명')
pattern['상품명'] = pattern['상품명'].astype(str)
pattern['상품명'] = pattern['상품명'].str.replace(r"['\[\]]", '')
pattern

In [ ]:
# 구매한 상품을 상품1,2,3,4에 할당

# '","'을 기준으로 분리하여 새로운 열에 할당
split_columns = pattern['상품명'].str.split('", "', expand=True)

# 새로운 열의 개수 확인
num_columns = len(split_columns.columns)

# '상품1', '상품2', ..., '상품N'으로 새로운 열 추가
pattern[[f'상품{i+1}' for i in range(num_columns)]] = split_columns

pattern

In [ ]:
# 2회 이상 구매한 기록만 남김
pattern = pattern[~pattern['상품2'].isna()]
pattern

,주문자ID,상품명,상품1,상품2,상품3,상품4,상품5
2,2341781572@k,"""Jemma bag_black"", ""Lucy bag_silver""","""Jemma bag_black","Lucy bag_silver""",None,None,None
21,2547512621@k,"""Lily bag_ivory, Jemma bag_ivory"", ""Glossy ven...","""Lily bag_ivory, Jemma bag_ivory","Glossy venti bag_ivory""",None,None,None
37,2626327844@k,"""Folnu bag_silver"", ""Popcorn bag_ivory""","""Folnu bag_silver","Popcorn bag_ivory""",None,None,None
53,2694172360@k,"""Glossy bag_black"", ""Glossy bag_black""","""Glossy bag_black","Glossy bag_black""",None,None,None
63,2710550319@k,"""Boheme bag_black"", ""Boheme bag_black""","""Boheme bag_black","Boheme bag_black""",None,None,None
...,...,...,...,...,...,...,...
8684,non-6133,"""Glossy bag_black"", ""Glossy bag_black""","""Glossy bag_black","Glossy bag_black""",None,None,None
8726,non-6175,"""Jemma bag_black"", ""Sendy bag_black""","""Jemma bag_black","Sendy bag_black""",None,None,None
8851,non-6300,"""Glossy bag_black"", ""Folnu bag_black""","""Glossy bag_black","Folnu bag_black""",None,None,None
8892,non-6341,"""Jemma bag_black"", ""Jemma bag_ivory""","""Jemma bag_black","Jemma bag_ivory""",None,None,None


In [ ]:
# "" 삭제
pattern['상품1'] = pattern['상품1'].str.replace('"', '')
pattern['상품2'] = pattern['상품2'].str.replace('"', '')
pattern['상품3'] = pattern['상품3'].str.replace('"', '')
pattern['상품4'] = pattern['상품4'].str.replace('"', '')
pattern['상품5'] = pattern['상품5'].str.replace('"', '')
pattern

In [ ]:
# 백업 파일 생성
pattern_plus = pattern

In [ ]:
# 새로운 행을 추가하는 함수 정의
new_rows_list = []

def split_and_create_rows(row):
    values1 = row['상품1'].split(', ') if pd.notna(row['상품1']) else []
    values2 = row['상품2'].split(', ') if pd.notna(row['상품2']) else []
    values3 = row['상품3'].split(', ') if pd.notna(row['상품3']) else []

    # 첫 번째 상품만 쪼개고 나머지는 그대로 가져오기
    if values1 and len(values1) > 1:
        new_row = row.copy()
        new_row['상품1'] = values1[0]
        new_rows_list.append(new_row)

        for i in range(1, len(values1)):
            new_row = row.copy()
            new_row['상품1'] = values1[i]
            new_row['상품2'] = None
            new_row['상품3'] = None
            new_rows_list.append(new_row)

    # 나머지 상품들은 그대로 가져오기
    max_len = max(len(values2), len(values3))
    for i in range(max_len):
        new_row = row.copy()
        if i < len(values2):
            new_row['상품2'] = values2[i]
        else:
            new_row['상품2'] = None

        if i < len(values3):
            new_row['상품3'] = values3[i]
        else:
            new_row['상품3'] = None

        new_rows_list.append(new_row)

# apply 함수를 사용하여 함수를 각 행에 적용
pattern.apply(split_and_create_rows, axis=1)

resulting_dataframe = pd.DataFrame(new_rows_list)


resulting_dataframe

In [ ]:
# 쉼표가 들어가는 행을 필터링하여 삭제
resulting_dataframe = resulting_dataframe[~(resulting_dataframe['상품1'].str.contains(',') | resulting_dataframe['상품1'].isna())]
resulting_dataframe = resulting_dataframe[~((resulting_dataframe['상품2'].str.contains(',') | resulting_dataframe['상품2'].isna()) & ~resulting_dataframe['상품2'].isna())]

# 결과 DataFrame 표시
resulting_dataframe

In [ ]:
# 같은 주문자 id를 가진 행의 '상품2'를 복사하여 넣어주기
# 예를 들어 a,b를 처음에 구매하고 다음번에 c를 구매했다면 우리는 그 사람을 a,c 로 산 것과 b,c로 산 것으로 나눔
# 가방을 구매한 뒤에 어떤 가방을 다음에 사는 지 확인하기 위해

resulting_dataframe['상품2'] = resulting_dataframe.groupby('주문자ID')['상품2'].transform(lambda x: x.fillna(method='ffill'))

# 결과 DataFrame 표시
resulting_dataframe

In [ ]:
# 구매한 기록이 1번만 있는 사람들 확인

# 상품2가 None인 행만 선택
filtered_dataframe = resulting_dataframe[resulting_dataframe['상품2'].isna()]

# 결과 DataFrame 표시
filtered_dataframe

In [ ]:
# 필요한 항목만 확인하기 위해 정리
want_columns = ['주문자ID','상품1','상품2']
resulting_dataframe = resulting_dataframe[want_columns]
resulting_dataframe

,주문자ID,상품1,상품2
0,2341781572@k,Jemma bag_black,Lucy bag_silver
1,2547512621@k,Lily bag_ivory,Glossy venti bag_ivory
2,2547512621@k,Jemma bag_ivory,Glossy venti bag_ivory
3,2626327844@k,Folnu bag_silver,Popcorn bag_ivory
4,2694172360@k,Glossy bag_black,Glossy bag_black
...,...,...,...
315,3049037133@k,Lily bag,Lucy bag
316,3049037133@k,keyring,keyring
317,3049037133@k,keyring,Lucy bag
318,3049037133@k,keyring,keyring


In [ ]:
# '상품1'과 '상품2'가 둘 다 'GV bag'인 경우 확인
confirm = resulting_dataframe[resulting_dataframe['상품1'].str.contains('Glossy venti bag') & resulting_dataframe['상품2'].str.contains('GV bag')]
confirm

In [ ]:
# 매번 G bag을 구매한 경우 확인

# '상품1'과 '상품2'가 둘 다 'G bag'인 행 선택
selected_rows = resulting_dataframe[(resulting_dataframe['상품1'] == 'G bag') & (resulting_dataframe['상품2'] == 'G bag')]

# 선택된 행에서 '주문자 ID' 열만 출력
selected_customer_ids = selected_rows['주문자ID']

# 결과 확인
print(selected_customer_ids)

4      2694172360@k
15     2794691795@k
29     2817370423@k
34     2817445782@k
35     2817851141@k
89     2913813337@k
112    2946158397@k
119    2956486130@k
134    2976651451@k
142    2986483175@k
143    2986624116@k
154    3019072671@k
159    3026691196@k
166    3060544442@k
181        non-1220
210        non-2562
236        non-5406
252        non-6082
255        non-6133
Name: 주문자ID, dtype: object


In [ ]:
# 상품1과 상품2 조합 확인
# ex ) a 가방과 d가방 구매한 인원은 23명

result = resulting_dataframe.groupby(['상품1', '상품2']).size().reset_index(name='cnt')
result_sorted = result.sort_values(by='cnt', ascending=False)
result_sorted.rename(columns={'cnt': '개수'}, inplace=True)
# result_sorted[result_sorted['개수'] > 1]
result_sorted[result_sorted['개수'] != 1]

In [ ]:
# 가방만 확인할 것이므로 키링이 들어간 경우는 제외
result_sorted = result_sorted[(result_sorted['상품1'] != 'keyring') & (result_sorted['상품2'] != 'keyring')]
result_sorted[result_sorted['개수'] != 1]

In [ ]:
result_sorted['개수'].sum()

246

In [ ]:
result_sorted.to_csv('result_sorted_12_15.csv', index=False)

In [ ]:
# 패턴이 없는 경우 상품1 가방 빈도수
pattern = pattern[~pattern['상품2'].isna()]
no_pattern = pattern.groupby('상품1').size().reset_index(name='개수')
no_pattern = no_pattern.sort_values(by='개수', ascending=False)
no_pattern

In [ ]:
# 패턴이 없는 경우 상품2 가방 빈도수
no_pattern = pattern.groupby('상품2').size().reset_index(name='개수')
no_pattern = no_pattern.sort_values(by='개수', ascending=False)
no_pattern

In [ ]:
# 패턴과 상관 없이 빈도 확인 : 상품1
yes_pattern = result_sorted
yes_pattern = yes_pattern.groupby('상품1').size().reset_index(name='개수')
yes_pattern = yes_pattern.sort_values(by='개수', ascending=False)
yes_pattern

In [ ]:
# 패턴과 상관 없이 빈도 확인 : 상품2
yes_pattern = result_sorted
yes_pattern = yes_pattern.groupby('상품2').size().reset_index(name='개수')
yes_pattern = yes_pattern.sort_values(by='개수', ascending=False)
yes_pattern

## 재구매율 : 사이즈

In [ ]:
# 가방의 사이즈 항목을 데이터에 추가
item = pd.read_csv('/content/제목 없는 스프레드시트 - 제품 상세정리본.csv')

selected_columns = ["제품명", "사이즈"]
result = item[selected_columns]
result['제품명'] = result['제품명'].str.lower()
result

In [ ]:
# resulting_dataframe 상품1과 상품2 소문자로 모두 변경
resulting_dataframe['상품1'] = resulting_dataframe['상품1'].str.lower()
resulting_dataframe['상품2'] = resulting_dataframe['상품2'].str.lower()

In [ ]:
# 상품1 사이즈 부여
size = pd.merge(resulting_dataframe, result, left_on='상품1', right_on='제품명', how='inner')
size.rename(columns={'사이즈': '상품1_사이즈'}, inplace=True)
sc = ['주문자ID','상품1','상품1_사이즈','상품2']
size = size[sc]
size

In [ ]:
# 상품2 사이즈 부여
size = pd.merge(size,result, left_on='상품2', right_on='제품명', how='inner')
size.rename(columns={'사이즈': '상품2_사이즈'}, inplace=True)
size

In [ ]:
# 필요한 항목만 추출
sc = ["주문자ID","상품1","상품1_사이즈","상품2","상품2_사이즈"]
size = size[sc]
size

,주문자ID,상품1,상품1_사이즈,상품2,상품2_사이즈
0,2341781572@k,jemma bag,빅,lucy bag,빅
1,3049037133@k,lily bag,미니,lucy bag,빅
2,2547512621@k,jemma bag,빅,glossy venti bag,빅
3,2895306997@k,jemma bag,빅,glossy venti bag,빅
4,2845335016@k,jemma bag,빅,glossy venti bag,빅
...,...,...,...,...,...
230,2931069415@k,glossy venti bag,빅,piko bag,미니
231,2952812982@k,endor bag,미디움,baguette bag,미디움
232,2952812982@k,endor bag,미디움,baguette bag,미디움
233,2873649420@k,twinkle bag,미니,boston bag,빅


In [ ]:
# 이용자들의 사이즈 별 구매 조합 확인
size_result = size.groupby(['상품1_사이즈', '상품2_사이즈']).size().reset_index(name='cnt')
size_result_sorted = size_result.sort_values(by='cnt', ascending=False)
size_result_sorted.rename(columns={'cnt': '개수'}, inplace=True)
#size_result_sorted[size_result_sorted['개수'] > 1]
size_result_sorted

,상품1_사이즈,상품2_사이즈,개수
8,빅,빅,119
6,빅,미니,21
0,미니,미니,20
2,미니,빅,20
5,미디움,빅,16
4,미디움,미디움,12
7,빅,미디움,11
1,미니,미디움,9
3,미디움,미니,7


## 재구매율 : 색상

In [ ]:
# 상품명_옵션에 색상이 적혀 있으므로 상품명_옵션이 담긴 데이터 다시 불러오기
color = color.groupby('주문자ID')['상품명'].agg(list).reset_index(name='상품명')
color['상품명'] = color['상품명'].astype(str)
color['상품명'] = color['상품명'].str.replace(r"['\[\]]", '')
color

In [ ]:
# '","'을 기준으로 분리하여 새로운 열에 할당
split_columns = color['상품명'].str.split('", "', expand=True)

# 새로운 열의 개수 확인
num_columns = len(split_columns.columns)

# '상품1', '상품2', ..., '상품N'으로 새로운 열 추가
color[[f'상품{i+1}' for i in range(num_columns)]] = split_columns

color

In [ ]:
# 재구매 기록이 없는 내역 제외
color = color[~color['상품2'].isna()]
color

In [ ]:
color['상품1'] = color['상품1'].str.replace('"', '')
color['상품2'] = color['상품2'].str.replace('"', '')
color['상품3'] = color['상품3'].str.replace('"', '')
color['상품4'] = color['상품4'].str.replace('"', '')
color['상품5'] = color['상품5'].str.replace('"', '')

In [ ]:
# 새로운 행을 추가하는 함수 정의
new_rows_list = []

def split_and_create_rows(row):
    values1 = row['상품1'].split(', ') if pd.notna(row['상품1']) else []
    values2 = row['상품2'].split(', ') if pd.notna(row['상품2']) else []
    values3 = row['상품3'].split(', ') if pd.notna(row['상품3']) else []

    # 첫 번째 상품만 쪼개고 나머지는 그대로 가져오기
    if values1 and len(values1) > 1:
        new_row = row.copy()
        new_row['상품1'] = values1[0]
        new_rows_list.append(new_row)

        for i in range(1, len(values1)):
            new_row = row.copy()
            new_row['상품1'] = values1[i]
            new_row['상품2'] = None
            new_row['상품3'] = None
            new_rows_list.append(new_row)

    # 나머지 상품들은 그대로 가져오기
    max_len = max(len(values2), len(values3))
    for i in range(max_len):
        new_row = row.copy()
        if i < len(values2):
            new_row['상품2'] = values2[i]
        else:
            new_row['상품2'] = None

        if i < len(values3):
            new_row['상품3'] = values3[i]
        else:
            new_row['상품3'] = None

        new_rows_list.append(new_row)

# apply 함수를 사용하여 함수를 각 행에 적용
color.apply(split_and_create_rows, axis=1)

# 결과로 나온 리스트를 DataFrame으로 변환
color_resulting_dataframe = pd.DataFrame(new_rows_list)

# 결과 DataFrame 표시
color_resulting_dataframe

# 같은 주문자 id를 가진 행의 '상품2'를 복사하여 넣어주기
color_resulting_dataframe['상품2'] = color_resulting_dataframe.groupby('주문자ID')['상품2'].transform(lambda x: x.fillna(method='ffill'))

# 결과 DataFrame 표시
color_resulting_dataframe

In [ ]:
want_columns = ['주문자ID','상품1','상품2']
color_resulting_dataframe = color_resulting_dataframe[want_columns]
color_resulting_dataframe

In [ ]:
# 쉼표가 들어가는 행을 필터링하여 삭제
color_resulting_dataframe = color_resulting_dataframe[~(color_resulting_dataframe['상품1'].str.contains(',') | color_resulting_dataframe['상품1'].isna())]
color_resulting_dataframe = color_resulting_dataframe[~((color_resulting_dataframe['상품2'].str.contains(',') | color_resulting_dataframe['상품2'].isna()) & ~color_resulting_dataframe['상품2'].isna())]

# 결과 DataFrame 표시
color_resulting_dataframe

In [ ]:
# 색상만 남기기
color_resulting_dataframe['상품1'] = color_resulting_dataframe['상품1'].str.extract('_(.*)')
color_resulting_dataframe['상품2'] = color_resulting_dataframe['상품2'].str.extract('_(.*)')
color_resulting_dataframe

,주문자ID,상품1,상품2
0,2341781572@k,black,silver
1,2547512621@k,ivory,ivory
2,2547512621@k,ivory,ivory
4,2626327844@k,silver,ivory
5,2694172360@k,black,black
...,...,...,...
327,2965822221@k,black,silver
328,2965822221@k,black,silver
329,2978171760@k,ivory,pink
330,3042318915@k,black,beige


In [ ]:
# 색상의 재구매 조합 확인
color_result = color_resulting_dataframe.groupby(['상품1', '상품2']).size().reset_index(name='cnt')
color_result_sorted = color_result.sort_values(by='cnt', ascending=False)
color_result_sorted.rename(columns={'cnt': '개수'}, inplace=True)
color_result_sorted.rename(columns={'상품1': '색상1'}, inplace=True)
color_result_sorted.rename(columns={'상품2': '색상2'}, inplace=True)
# size_result_sorted[size_result_sorted['cnt'] > 1]
color_result_sorted

,색상1,색상2,개수
1,black,black,65
5,black,ivory,33
17,ivory,black,31
20,ivory,ivory,17
7,black,silver,13
3,black,brown,12
22,ivory,silver,9
13,brown,black,7
27,silver,black,7
30,silver,silver,5


In [ ]:
color_result_sorted['개수'].sum()

237